In this second step we'll be cleaning weekly_returns from the issues we found in the data validations steps.

We could create a dictionary or new dataframe with all the missing values and then merge it into the current dataframe but given the low amount of data needed to be added we can also just .loc it in with negative indexing

In [79]:
import pandas as pd
import numpy as np

weeklyreturnsdf = pd.read_csv('DataSets/weekly_returns.csv')
tickerattridf = pd.read_csv('DataSets/ticker_attributes.csv')

weeklyreturnsdf.loc[-4] = ['5/6/2011', 0, -3.500018]
weeklyreturnsdf.loc[-3] = ['5/13/2011', 0, -2.756460]
weeklyreturnsdf.loc[-2] = ['5/20/2011', 0, 0.319715]
weeklyreturnsdf.loc[-1] = ['5/27/2011', 0, 1.444635]
weeklyreturnsdf.loc[3650] = ['4/30/2021', 6, 0.023186]
weeklyreturnsdf.index = weeklyreturnsdf.index + 4  # shifting index
weeklyreturnsdf = weeklyreturnsdf.sort_index()  # sorting by index

print(weeklyreturnsdf.head(n=10))
print("\n")
print(weeklyreturnsdf.info())

        date  ticker_id  weekly_return
0   5/6/2011          0      -3.500018
1  5/13/2011          0      -2.756460
2  5/20/2011          0       0.319715
3  5/27/2011          0       1.444635
4   6/3/2011          0       0.251305
5  6/10/2011          0      -2.924585
6  6/17/2011          0      -1.463312
7  6/24/2011          0       0.373256
8   7/1/2011          0       5.846155
9   7/8/2011          0      -0.477576


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3654 entries, 0 to 3654
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           3654 non-null   object 
 1   ticker_id      3654 non-null   int64  
 2   weekly_return  3654 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 114.2+ KB
None


Now we just replace the -100% from ticker 4 on 5/10/2019 with  0.304620


In [80]:
weeklyreturnsdf.loc[((weeklyreturnsdf.date == '5/10/2019') & (weeklyreturnsdf.ticker_id == 4)),'weekly_return']=0.304620
print(weeklyreturnsdf.loc[((weeklyreturnsdf.date == '5/10/2019') & (weeklyreturnsdf.ticker_id == 4))])
print("\n")
print(weeklyreturnsdf.describe())
print("\n")

weeklyreturnsdf['date'] =  pd.to_datetime(weeklyreturnsdf['date'], format='%m/%d/%Y')
tickerdatesdf = weeklyreturnsdf.groupby(['ticker_id'])
tickerdatesdf2 = tickerdatesdf.agg(Minimum_Date=('date', np.min), Maximum_Date=('date', np.max))
print(tickerdatesdf2)

           date  ticker_id  weekly_return
2506  5/10/2019          4        0.30462


         ticker_id  weekly_return
count  3654.000000    3654.000000
mean      3.000000       0.151485
std       2.000274       2.205844
min       0.000000     -16.636300
25%       1.000000      -0.420158
50%       3.000000       0.074637
75%       5.000000       0.811958
max       6.000000      44.000000


          Minimum_Date Maximum_Date
ticker_id                          
0           2011-05-06   2021-04-30
1           2011-05-06   2021-04-30
2           2011-05-06   2021-04-30
3           2011-05-06   2021-04-30
4           2011-05-06   2021-04-30
5           2011-05-06   2021-04-30
6           2011-05-06   2021-04-30


in order to better use this data set we're also going to move ticker_id's to their stock tickers and move them to columns so we can use our dates as an index in future dataframes. We're also going to divide all of the returns by 100 in order to return them to a percentage.

In [81]:
weeklyreturnsdf = pd.merge(weeklyreturnsdf,tickerattridf[['ticker_id','ticker']],on='ticker_id', how='left')
weeklyreturnsdf = weeklyreturnsdf.drop(columns=['ticker_id'])
weeklyreturnsdf = pd.pivot_table(weeklyreturnsdf, values='weekly_return', index=['date'],
                    columns=['ticker'])
weeklyreturnsdf = weeklyreturnsdf.div(100)
print(weeklyreturnsdf)

ticker           AGG       EEM       EFA       IJR       JNK       SHY  \
date                                                                     
2011-05-06  0.004848 -0.035000 -0.029310 -0.030733  0.001847  0.001024   
2011-05-13  0.001219 -0.027565 -0.016071  0.007927 -0.002210  0.000475   
2011-05-20  0.001873  0.003197 -0.002145 -0.008949  0.004430  0.000594   
2011-05-27  0.004113  0.014446  0.005291  0.008893 -0.000980  0.001543   
2011-06-03  0.001887  0.002513  0.002138 -0.032818 -0.011500  0.000427   
...              ...       ...       ...       ...       ...       ...   
2021-04-01  0.002653  0.011076  0.003394  0.007320  0.004751 -0.000587   
2021-04-09  0.001228 -0.005756  0.014702 -0.005268  0.002301  0.000116   
2021-04-16  0.003153  0.014939  0.016669  0.011506  0.001010  0.000232   
2021-04-23  0.001833  0.005152 -0.002775  0.001264  0.001101  0.000000   
2021-04-30 -0.002266 -0.011898 -0.012141 -0.003336  0.000825  0.000232   

ticker           SPY  
date          

Now that the data has been cleaned, is logical, and aligns on dates we can output this dataframe to a clean .csv file for our portfolio optimization


In [82]:
weeklyreturnsdf.to_csv("DataSets/weekly_returns_clean.csv",index=True)